In [ ]:
from federated import agglomerate
import torch
from time import sleep
from server import move_left, move_forward, move_right, stop_robot
from server import display_camera_stream
from server import get_state, set_drive, get_weights, set_weights

In [ ]:
def x_y_to_action(x=float, y=float):
    """
    X and Y are clamped to between 0 and 1, -1, -1 if not applicable
    """
    if x < 0 or y < 0:
        return (0, 0.2)

    if x < 0.5:
        direction = -1
    else:
        direction = 1
    turn_speed = abs(x-0.5)/0.5

    forward_speed = abs(y-0.5)/0.5

    return (forward_speed, turn_speed*direction)

In [ ]:
def PSO(colour_cords: list[dict], colors):
    best_score = (-1, -1)
    best_robot = 0
    acts = [(0,0)]*len(colour_cords)
    for index, robot in enumerate(colour_cords): 
        box_cords = robot["red"]
        if box_cords[0]>= 0 and box_cords[1]>= 0:
            score = abs((0.5, 0.5) - box_cords)
        else:
            score = (-1, -1)
        acts[index] = x_y_to_action(*box_cords)
        
        if score > best_score:
            best_robot = index
    
    if len(colour_cords) > 1 and best_score > (0, 0):
        for index, robot in enumerate(colour_cords): 
            if index == best_robot:
                continue
            best_robot_colour = colors[best_robot]
            random_int = 0.5
            acts[index] = random_int * acts[index] + (1-random_int) * x_y_to_action(*robot[best_robot_colour]) 

    return acts

In [ ]:
# Test connection
IPS = ["194.47.156.140", "194.47.156.22"]
COLOURS = ["blue", "yellow"]
for i in range(0, 100):
    states = []
    for index, IP in enumerate(IPS):
        states.append(get_state(IP))
states

In [ ]:
weights = [None]*len(IPS)
states = [(0,0)]*len(IPS)
for i in range(0, 1):
    states = []
    for index, IP in enumerate(IPS):
        states.append(get_state(IP))

    descisions = PSO(states, COLOURS)

    for index, IP in enumerate(IPS):
        move_robot =  set_drive(IP, descisions[index][0], descisions[index][1])

    sleep(1) 

    for index, IP in enumerate(IPS):
        stop_robot(IP)
    
    if i % 10 == 0:
        for index, IP in enumerate(IPS):
            weights[index] = get_weights(IP)

        agg_weight = agglomerate(weights)
        
        for index, IP in enumerate(IPS):
            send_weights = set_weights(IP, agg_weight)
            pass

for index, IP in enumerate(IPS):
    stop_robot = stop_robot(IP)
    weights[index] = get_weights(IP)

agg_weight = agglomerate(weights)
torch.save(agg_weight, "model.pth")

In [ ]:
states[0]["red"]

In [ ]:
weights[0]